In [1]:
import pandas as pd
train_data = pd.read_csv(r"C:\Users\Suvir Gupta\PycharmProjects\Projects\word2vec\nueral net\data\labeledTrainData.tsv",
                         sep = "\t", header= (0))

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
def preprocessing(reviews):
## remove the Html Tags
    review_text = BeautifulSoup(reviews).get_text()
## Remove non letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
## set all words to lower digits
    review_text = review_text.lower().split()
## remove stopwords
    stopword = set(stopwords.words("english"))
    words = [w for w in review_text if w not in stopword]
### lemmatization of the words to reduce them to their standard foramat
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(w) for w in words]
    return words

In [7]:
vocabulary= [preprocessing(review) for review in train_data["review"]]

C:\Users\Suvir Gupta\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Suvir Gupta\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [16]:
vocabulary_flat = [word for wd_list in vocabulary for word in wd_list]

In [20]:
vocabulary_size = 50000

In [22]:
import collections
def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

In [23]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary_flat,
                                                            vocabulary_size)
# del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 16469], ('movie', 51694), ('film', 47034), ('one', 27741), ('like', 20740)]
Sample data [408, 77, 141, 8138, 519, 2372, 123, 65, 871, 451] ['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary']


In [26]:
del vocabulary_flat

NameError: name 'vocabulary_flat' is not defined

In [35]:
data_index = 0
import numpy as np
import random
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    if data_index == len(data):
      # buffer[:] = data[:span]
        for word in data[:span]:
            buffer.append(word)
        data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

77 going -> 408 stuff
77 going -> 141 moment
141 moment -> 8138 mj
141 moment -> 77 going
8138 mj -> 141 moment
8138 mj -> 519 started
519 started -> 8138 mj
519 started -> 2372 listening


In [36]:
import tensorflow as tf
import math
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [37]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  289.227996826


Nearest to work: williamsburg, slingshot, joslyn, everglades, properly, jud, third, bumptious,
Nearest to fact: anglophile, providence, respected, babcock, unflattering, vampyres, wonderfalls, jasmin,
Nearest to right: founded, imitates, hehe, balderdash, lonny, dreichness, kristen, dairy,
Nearest to seems: mammet, rimmed, magalh, supported, inasmuch, absence, slat, cheerleading,
Nearest to horror: jerrine, cincinnati, noon, inheritor, davison, barbour, mcintosh, egomaniac,
Nearest to still: sighing, control, straighten, memorable, perpetrator, flom, ashtray, rehman,
Nearest to minute: scotland, insightfully, profitability, morphin, wopr, wedlock, snowed, mclovins,
Nearest to comedy: disenchantment, bryant, bachman, behead, unpopularity, elektra, computer, creole,
Nearest to like: recreated, meagre, interfere, paternity, sheik, neelix, miscarrage, pah,
Nearest to acting: nailbiters, morrill, breathable, broon, forsythe, matchbox, elongated, signifying,
Nearest to world: faridany, camou

Nearest to woman: wilkinson, civilizing, alert, rebellion, costa, uomo, rye, cobb,


Average loss at step  2000 :  124.264425044


Average loss at step  4000 :  56.5099060798


...

Average loss at step  94000 :  4.95027623868


Average loss at step  96000 :  4.95869459414


Average loss at step  98000 :  4.92501607323


Average loss at step  100000 :  4.90524826884


Nearest to work: done, renn, argonne, sayori, taj, although, show, paine,
Nearest to fact: also, however, say, respected, point, reason, genma, really,
Nearest to right: booker, going, letty, catalunya, tyrannous, went, growl, jogging,
Nearest to seems: seem, look, seemed, felt, feel, letty, happens, kaakha,
Nearest to horror: slasher, classic, thriller, doorstep, muddy, filled, egomaniac, ernst,
Nearest to still: also, really, however, always, actually, even, may, letty,
Nearest to minute: hour, time, scotland, episode, week, year, profitability, tbn,
Nearest to comedy: gravitation, medical, thriller, deadbeat, gauntlet, utter, film, behead,
Nearest to like: reviewed, empowerment, think, morand, booker, sasquatch, know, made,
Nearest to acting: script, performance, direction, dialogue, cinematography, writing, actor, due,
Nearest to world: dip, life, event, lv, prevail, volckman, therapy, invested,
Nearest to got: get, getting, alecia, ingenious, alluded, watched, grahame, hesitancy,


In [43]:
%matplotlib inline
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in range(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')